In [1]:
import onnx
import numpy as np
from qonnx.util.basic import qonnx_make_model
from finn.util.visualization import showInNetron,showSrc
import onnxruntime as rt
from qonnx.util.basic import qonnx_make_model
from onnx.helper import make_tensor_value_info, make_node, make_graph, make_model, make_tensor
from onnx import numpy_helper
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
import finn.core.onnx_exec as oxe
import torch
from brevitas.nn import QuantReLU, QuantIdentity
from finn.transformation.streamline import Streamline
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline import RoundAndClipThresholds
from qonnx.core.datatype import DataType
from qonnx.transformation.qcdq_to_qonnx import QCDQToQuant
import finn.core.onnx_exec as oxe

In [2]:
# id_path = "quant_threshold.onnx"
#id_path = "quant_threshold_clip.onnx"
#id_path = "lstm-gate-sigmoid.onnx"
#id_path = "lstm-gate-tanh.onnx"
#id_path = "quantize-lstm-full-graph-test.onnx"
id_path = "quantize-lstm-full-graph-test-no-identity.onnx"
model = ModelWrapper(id_path)
model = model.transform(QCDQToQuant())
model = model.transform(InferShapes())
model.save("./quant_threshold_transform.onnx") 

In [3]:
#showInNetron('./lstm-gate-sigmoid.onnx',localhost_url='xirxlabs53')#,localhost_url='xirxlabs53'
#showInNetron('./lstm-gate-tanh.onnx')#,localhost_url='xirxlabs53'
showInNetron('./quantize-lstm-full-graph-test.onnx')#,,localhost_url='xirxlabs53'

Serving './quantize-lstm-full-graph-test.onnx' at http://0.0.0.0:5902


In [4]:
showInNetron('./quant_threshold_transform.onnx')#,localhost_url='xirxlabs53'

Stopping http://0.0.0.0:5902
Serving './quant_threshold_transform.onnx' at http://0.0.0.0:5902


In [5]:
in_X = np.array([0,0,0,0,0,0,0,0,0,50],dtype=np.float32)
#in_X = np.array([1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
#in_X = np.array([1,2,3,4,5,-6,-7,-8,-9,-10],dtype=np.float32)
#in_X = np.array([0,0,0,0,0,0,0,0,0,1],dtype=np.float32)
in_X = in_X.reshape([10,1])
in_h_t_1 = np.zeros([20,1],dtype=np.float32).reshape([20,1])
input_dict = {}
input_dict["X"] = in_X
input_dict["h_t-1"] = in_h_t_1

output_dict = oxe.execute_onnx(model, input_dict,return_full_exec_context=True)
#output_dict = oxe.execute_onnx(model, input_dict)
print(output_dict)
#values = np.array(output_dict.values())
QONNX_out = np.array(output_dict.get("h_t"))

{'h_t-1': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32), 'c_t-1': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32), 'X': array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [50.]], dtype=float32), 'h_t': array([[-0.046875 ],
       [ 0.0078125],
       [ 0.03125  ],
       [-0.0234375],
       [-0.015625 ],
       [ 0.046875 ],
       [ 0.0625   ],
       [ 0.0078125],
       [ 0.0390625],
       [-0.0546875],
       [ 0.015625 ],
       [-0.03125  ],
      

In [6]:
finn_model = model.transform(ConvertQONNXtoFINN())
finn_model.save("./quant_threshold_finn_onnx.onnx") 

In [7]:
showInNetron('./quant_threshold_finn_onnx.onnx')#,,localhost_url='xirxlabs53'

Stopping http://0.0.0.0:5902
Serving './quant_threshold_finn_onnx.onnx' at http://0.0.0.0:5902


In [8]:
#Doing the FINN-ONNX ececution here to compare against the onnx graph execution
in_X = np.array([0,0,0,0,0,0,0,0,0,50],dtype=np.float32)
#in_X = np.array([1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
#in_X = np.array([1,2,3,4,5,-6,-7,-8,-9,-10],dtype=np.float32)
#in_X = np.array([0,0,0,0,0,0,0,0,0,1],dtype=np.float32)
in_X = in_X.reshape([10,1])
in_h_t_1 = np.zeros([20,1],dtype=np.float32).reshape([20,1])
input_dict = {}
input_dict["X"] = in_X
input_dict["h_t-1"] = in_h_t_1

output_dict_finn = oxe.execute_onnx(finn_model, input_dict,return_full_exec_context=True)
print(output_dict_finn)
FINN_ONNX_out = np.array(output_dict_finn.get("h_t"))

/scratch/users/khandelw/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor eooXNj can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/scratch/users/khandelw/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor gkvAOI can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/scratch/users/khandelw/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor W4QEzJ can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/scratch/users/khandelw/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor K6VIGl can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/scratch/users/khandelw/

{'h_t-1': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32), 'c_t-1': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32), 'X': array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [50.]], dtype=float32), 'h_t': array([[-0.046875 ],
       [ 0.0078125],
       [ 0.0234375],
       [-0.0234375],
       [-0.015625 ],
       [ 0.046875 ],
       [ 0.0546875],
       [ 0.0078125],
       [ 0.0390625],
       [-0.0546875],
       [ 0.015625 ],
       [-0.03125  ],
      

/scratch/users/khandelw/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor wGCSf6 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/scratch/users/khandelw/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor 2sAOzQ can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(


In [9]:
x = np.array(output_dict.get("i_t_dql1"))
print(x)
x1 = np.array(output_dict_finn.get("i_t_dql1"))
print(x1)

[[ 0.1484375]
 [-0.1015625]
 [-0.265625 ]
 [-0.0234375]
 [-0.1953125]
 [ 0.0546875]
 [-0.34375  ]
 [ 0.0078125]
 [-0.234375 ]
 [-0.375    ]
 [-0.015625 ]
 [ 0.1171875]
 [ 0.015625 ]
 [ 0.2578125]
 [ 0.25     ]
 [ 0.2109375]
 [ 0.09375  ]
 [-0.28125  ]
 [ 0.       ]
 [-0.2421875]]
[[ 0.1484375]
 [-0.1015625]
 [-0.265625 ]
 [-0.0234375]
 [-0.1953125]
 [ 0.0546875]
 [-0.34375  ]
 [ 0.0078125]
 [-0.234375 ]
 [-0.375    ]
 [-0.015625 ]
 [ 0.1171875]
 [ 0.015625 ]
 [ 0.2578125]
 [ 0.25     ]
 [ 0.2109375]
 [ 0.09375  ]
 [-0.28125  ]
 [ 0.       ]
 [-0.2421875]]


In [10]:
#MT = np.array(output_dict.get("FqSx1q"))
M1 = np.array(output_dict.get("c_t_dql2"))
#print(M1)
M1F = np.array(output_dict_finn.get("c_t_dql2"))
#print(M1F)
diff = M1F - M1
print(diff)

[[ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [-0.0078125]
 [-0.0078125]
 [ 0.       ]
 [-0.0078125]
 [-0.0078125]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]
 [-0.0078125]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]]


In [11]:
#Comparing FINN-ONNX and QONNX output
#print(FINN_ONNX_out.shape)
scale = 0.0078125
difference = FINN_ONNX_out - QONNX_out
difference = difference/scale
print(difference)

[[ 0.]
 [ 0.]
 [-1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]


In [12]:
from finn.transformation.streamline import Streamline
showSrc(Streamline)

class Streamline(Transformation):
    """Apply the streamlining transform, see arXiv:1709.04060."""

    def apply(self, model):
        streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),
            MoveMulPastMaxPool(),
            AbsorbAddIntoMultiThreshold(),
            FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
        for tr

In [13]:
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants
#streamlined_model = finn_model.transform(Streamline())
finn_model = finn_model.transform(InferShapes())
finn_model = finn_model.transform(FoldConstants())
finn_model = finn_model.transform(GiveUniqueNodeNames())
finn_model = finn_model.transform(GiveReadableTensorNames())
finn_model = finn_model.transform(InferDataTypes())
finn_model = finn_model.transform(RemoveStaticGraphInputs())
finn_model.save('./tidy_up_model.onnx')

In [14]:
showInNetron('./tidy_up_model.onnx')#,,localhost_url='xirxlabs53'

Stopping http://0.0.0.0:5902
Serving './tidy_up_model.onnx' at http://0.0.0.0:5902


In [15]:
showSrc(Streamline)

class Streamline(Transformation):
    """Apply the streamlining transform, see arXiv:1709.04060."""

    def apply(self, model):
        streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),
            MoveMulPastMaxPool(),
            AbsorbAddIntoMultiThreshold(),
            FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
        for tr

In [16]:
from qonnx.transformation.base import Transformation
from qonnx.transformation.batchnorm_to_affine import BatchNormToAffine
from qonnx.transformation.general import (
    ConvertDivToMul,
    ConvertSubToAdd,
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
)
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.remove import RemoveIdentityOps

from finn.transformation.streamline.absorb import (
    Absorb1BitMulIntoConv,
    Absorb1BitMulIntoMatMul,
    AbsorbAddIntoMultiThreshold,
    AbsorbMulIntoMultiThreshold,
    AbsorbSignBiasIntoMultiThreshold,
    FactorOutMulSignMagnitude,
)
from finn.transformation.streamline.collapse_repeated import (
    CollapseRepeatedAdd,
    CollapseRepeatedMul,
)
from finn.transformation.streamline.reorder import (
    MoveAddPastConv,
    MoveAddPastMul,
    MoveMulPastMaxPool,
    MoveScalarAddPastMatMul,
    MoveScalarLinearPastInvariants,
    MoveScalarMulPastConv,
    MoveScalarMulPastMatMul,
)
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from finn.transformation.streamline.sign_to_thres import ConvertSignToThres

In [17]:
streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            #CollapseRepeatedMul(),
            MoveMulPastMaxPool(),
            AbsorbAddIntoMultiThreshold(),
            #FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
for trn in streamline_transformations:
    print('Transformation = ',trn)
    finn_model = finn_model.transform(trn)
    finn_model = finn_model.transform(RemoveIdentityOps())
    finn_model = finn_model.transform(GiveUniqueNodeNames())
    finn_model = finn_model.transform(GiveReadableTensorNames())
    finn_model = finn_model.transform(InferDataTypes())

Transformation =  <qonnx.transformation.general.ConvertSubToAdd object at 0x7fd68d440d30>
Transformation =  <qonnx.transformation.general.ConvertDivToMul object at 0x7fd68d440d00>
Transformation =  <qonnx.transformation.batchnorm_to_affine.BatchNormToAffine object at 0x7fd68d4434c0>
Transformation =  <finn.transformation.streamline.sign_to_thres.ConvertSignToThres object at 0x7fd68d441750>
Transformation =  <finn.transformation.streamline.reorder.MoveMulPastMaxPool object at 0x7fd68d440430>
Transformation =  <finn.transformation.streamline.reorder.MoveScalarLinearPastInvariants object at 0x7fd68d440a30>
Transformation =  <finn.transformation.streamline.absorb.AbsorbSignBiasIntoMultiThreshold object at 0x7fd68d4410f0>
Transformation =  <finn.transformation.streamline.reorder.MoveAddPastMul object at 0x7fd68d441120>
Transformation =  <finn.transformation.streamline.reorder.MoveScalarAddPastMatMul object at 0x7fd68d4423b0>
Transformation =  <finn.transformation.streamline.reorder.MoveAddP

AssertionError: Initializer for mul weights is not set.

In [ ]:
finn_model.save('./streamlined_model.onnx')
#showInNetron('./streamlined_model.onnx')#,,localhost_url='xirxlabs53'

Okay so till now only the add nodes have been absorbed into the multithresholding node. 
<br>In the streamline operation, three transformations were not applied on the graph.
<br> 1. <b>CollapseRepeatedMul()</b> : This is because the transformation does not allow one of the inputs in the 'Mul' node to be an input. We know why this transformation does not work. It needs to be customized or additional transformations need to be written to make it part of the streamlining process.
<br> 2. <b>FactorOutMulSignMagnitude()</b> : This error is again caused by one of the inputs to the MUL node being an input. The error is in absorb.py file.
<br> 3. <b>AbsorbMulIntoMultiThreshold()</b> : This error is again caused by one of the inputs to the MUL node being an input. The error is in absorb.py file.

<br> All the three errors are caused by the fact that one of the inputs to the 'Mul' Node is an input. The error I get for all the three transformations is this : 'AssertionError: Initializer for mul weights is not set.'
